In [3]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using device:", device)

pin_memory = use_cuda
use_amp = use_cuda
if use_amp:
    from torch.cuda.amp import autocast, GradScaler
    scaler = GradScaler()


Using device: cpu


In [4]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


In [5]:

data_dir = "./data"  # 폴더 구조: ./data/class_name/image.jpg

dataset = datasets.ImageFolder(data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4, pin_memory=pin_memory)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4, pin_memory=pin_memory)
print("Train size:", len(train_set), "Val size:", len(val_set))


Train size: 2562 Val size: 641


In [6]:

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # 클래스 수에 맞게 조정
model = model.to(device)


C:\Users\USER\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [8]:

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        if use_amp:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    print(f"Train Loss: {running_loss / len(train_loader):.4f}, Accuracy: {correct / total * 100:.2f}%")


Epoch 1/10: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [02:08<00:00,  3.14s/it]


Train Loss: 0.0372, Accuracy: 98.44%


Epoch 2/10: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [02:08<00:00,  3.13s/it]


Train Loss: 0.0187, Accuracy: 99.88%


Epoch 3/10: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [02:07<00:00,  3.10s/it]


...

Train Loss: 0.0100, Accuracy: 99.92%


Epoch 9/10: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [02:07<00:00,  3.11s/it]


Train Loss: 0.0060, Accuracy: 99.92%


Epoch 10/10: 100%|█████████████████████████████████████████████████████████████████████| 41/41 [02:09<00:00,  3.15s/it]

Train Loss: 0.0054, Accuracy: 99.96%


In [9]:

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct / total * 100:.2f}%")


Validation Accuracy: 100.00%


In [ ]:
# 8. 학습 이후 예측 결과 10개 테스트 출력 및 결과 확인


class_names = ['Real', 'AI']
model.eval()
num_images = 10
cols = 5
rows = (num_images + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 2.2, rows * 2.2))
shown = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images.to(device))
        _, preds = torch.max(outputs, 1)
        for j in range(images.size(0)):
            if shown >= num_images: break
            
            # Convert image tensor to numpy array for plotting
            img_np = images[j].permute(1, 2, 0).numpy()
            ax = axes[shown // cols, shown % cols]
            ax.imshow(img_np)
            ax.set_title(f"GT: {class_names[labels[j]]}\nPR: {class_names[preds[j]]}", fontsize=6)
            ax.axis('off')
            shown += 1
        if shown >= num_images: break

# 남은 서브플롯을 빈 그래프로 채우기
for i in range(shown, rows * cols):
    axes[i // cols, i % cols].axis('off')

plt.tight_layout()
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..0.92941177].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..1.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.9607843..0.99215686].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..0.8117647].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.99215686..0.99215686].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..0.85882354].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..0.9843137].
Clipping input d

In [ ]:
from PIL import Image
import torch.nn.functional as F

# 모델을 활용하여 이미지 예측 및 결과 도출
"""이미지 파일을 로드하고 예측을 수행하며 결과를 표시합니다"""
def predict_image(image_path, model, transform, class_names=['Real', 'AI'], device='cpu'):
    model.eval()
    model = model.to(device)

    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"ERROR: file open: {e}")
        return

    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        prob = F.softmax(output, dim=1)
        pred_class = torch.argmax(prob, dim=1).item()
        confidence = prob[0, pred_class].item()

    print(f"파일명: {os.path.basename(image_path)}")
    print(f"예측 결과: {class_names[pred_class]}")
    print(f"확신도: {confidence * 100:.2f}%")
    print(f"클래스별 확률: {[f'{name}: {p:.2%}' for name, p in zip(class_names, prob[0].cpu().numpy())]}")

    # 이미지 시각화
    plt.imshow(np.array(image))
    plt.title(f"Prediction: {class_names[pred_class]} ({confidence*100:.1f}%)")
    plt.axis('off')
    plt.show()

In [ ]:
import numpy as np

# image_path = './karina.jpg'
# image_path = './park.jpg'
# image_path = './kim3.jpeg'
# image_path = './ai2.jpg'
# image_path = './ai3.png'
# image_path = './karina3.jpeg'
# image_path = './do.jpg'
# image_path = './kim.jpeg'
# image_path = './winter.jpg'
image_path = './test/KakaoTalk_20250606_154526730.png'
# image_path = './hong.jpg'
# image_path = './chacha.jpg'
# image_path = './lee.jpeg'
# Perform inference
predict_image(image_path, model, transform_test)
